In [23]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import torch
from PIL import Image
from numpy import asarray

import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
)

import logging
logger = logging.getLogger(__name__)

In [2]:
# after process data structure for each modality
# acoustic = [[[feature at time 1], [feature at time 2], [...]], [...]]
# linguistic = [[[token_1, token_2, ... at time 1], [token_1, token_2, ... at time 2], [...]], [...]]
# visual = [[[feature at time 1], [feature at time 2], [...]], [...]]

In [3]:
modalities_data_dir = "../../SENDv1-data/features/Train/"

In [4]:
preprocess = {
    'acoustic': lambda df : df.loc[:,' F0semitoneFrom27.5Hz_sma3nz_amean':' equivalentSoundLevel_dBp'],
    'acoustic_timer': lambda df : df.loc[:,' frameTime'],
    'linguistic': lambda df : df.loc[:,'word'],
    'linguistic_timer': lambda df : df.loc[:,'time-offset'],
}

class InputFeature:
    
    def __init__(
        self, video_id="",
        acoustic_feature=[],
        linguistic_feature=[],
        visual_feature=[],
        labels=[],
    ):
        self.video_id = video_id
        self.acoustic_feature = acoustic_feature
        self.linguistic_feature = linguistic_feature
        self.visual_feature = visual_feature
        self.labels = labels

In [41]:
def preprocess_SEND_files(
    data_dir,
    time_window_in_sec=5.0,
    modality_dir_map = {"acoustic": "acoustic-egemaps",  
                        "linguistic": "linguistic-word-level", # we don't load features
                        "visual": "image-raw", # image is nested
                       },
    keep_first=True,
    linguistic_tokenizer=None,
):
    # basically, let us gett all the video ids?
    a_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["acoustic"])) 
             if isfile(os.path.join(data_dir, modality_dir_map["acoustic"], f))]
    l_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["linguistic"])) 
             if isfile(os.path.join(data_dir, modality_dir_map["linguistic"], f))]
    v_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["visual"])) 
             if f != ".DS_Store"]
    assert len(a_ids) == len(l_ids) and len(l_ids) == len(v_ids)
    assert len(set(a_ids).intersection(set(l_ids))) == len(l_ids)
    assert len(set(a_ids).intersection(set(v_ids))) == len(v_ids)
    
    # We need the first pass for linguistic modality process?
    
    for video_id in a_ids: # pick any one!
        
        # acoustic features process
        a_file = os.path.join(data_dir, modality_dir_map["acoustic"], f"{video_id}_acousticFeatures.csv")
        a_df = pd.read_csv(a_file)
        a_features = np.array(preprocess["acoustic"](a_df))
        a_timestamps = np.array(preprocess["acoustic_timer"](a_df))
        assert a_features.shape[0] == a_timestamps.shape[0]
        # sample based on interval
        current_time = 0.0
        keep_first = keep_first
        sampled_a_features = []
        sampled_a_timestamps  = []
        for i in range(0, a_timestamps.shape[0]):
            if keep_first:
                sampled_a_features += [a_features[i]]
                sampled_a_timestamps+= [a_timestamps[i]]
                keep_first = False
            if a_timestamps[i] >= current_time+time_window_in_sec:
                sampled_a_features += [a_features[i]]
                sampled_a_timestamps+= [a_timestamps[i]]
                current_time += time_window_in_sec
        sampled_a_features = np.array(sampled_a_features)
        sampled_a_timestamps = np.array(sampled_a_timestamps)
        assert sampled_a_features.shape[0] == sampled_a_timestamps.shape[0]
        
        # linguistic features process
        l_file = os.path.join(data_dir, modality_dir_map["linguistic"], f"{video_id}_aligned.tsv")
        l_df = pd.read_csv(l_file, sep='\t')
        l_words = np.array(preprocess["linguistic"](l_df))
        l_words = [w.strip().lower() for w in l_words]
        l_timestamps = np.array(preprocess["linguistic_timer"](l_df))
        assert len(l_words) == l_timestamps.shape[0]
        # sample based on interval
        current_time = 0.0
        keep_first = keep_first
        sampled_l_words = [] # different from other modality, it is essentially a list of list!
        sampled_l_timestamps = []
        tmp_words = []
        for i in range(0, l_timestamps.shape[0]):
            if keep_first:
                sampled_l_words += [[]]
                sampled_l_timestamps += [0.0]
                keep_first = False
            if l_timestamps[i] >= current_time+time_window_in_sec:
                sampled_l_words.append(tmp_words)
                this_timestampe = current_time+time_window_in_sec
                sampled_l_timestamps += [this_timestampe]
                tmp_words = [l_words[i]] # reinit the buffer
                current_time += time_window_in_sec
                continue
            tmp_words += [l_words[i]]
        # overflow
        if len(tmp_words) > 0:
            sampled_l_words.append(tmp_words)
            this_timestampe = current_time+time_window_in_sec
            sampled_l_timestamps += [this_timestampe]
        sampled_l_timestamps = np.array(sampled_l_timestamps)
        sampled_l_token_ids = []
        for window_words in sampled_l_words:
            token_ids = linguistic_tokenizer.convert_tokens_to_ids(window_words)
            sampled_l_token_ids += [token_ids]
        assert len(sampled_l_words) == sampled_l_timestamps.shape[0]
        assert len(sampled_l_token_ids) == sampled_l_timestamps.shape[0]
        print(sampled_l_token_ids)
        
        # visual features process
        # for visual, we actually need to active control what image we load, we
        # cannot just load all images, it will below memory.
        fps=30 # We may need to dynamically figure out this number?
        frame_names = []
        for f in listdir(os.path.join(data_dir, modality_dir_map["visual"], video_id)):
            if ".jpg" in f:
                frame_names += [(int(f.split("_")[0][5:])*(1.0/fps), f)]
        frame_names.sort(key=lambda x:x[0])
        sampled_frames = []
        current_time = 0.0
        keep_first = keep_first
        for f in frame_names:
            if keep_first:
                sampled_frames += [f]
                keep_first = False
            if f[0] >= current_time+time_window_in_sec:
                sampled_frames += [f]
                current_time += time_window_in_sec
        v_images = []
        v_timestamps = []
        for f in sampled_frames:
            f_path = os.path.join(data_dir, modality_dir_map["visual"], video_id, f[1])
            f_image = Image.open(f_path)
            f_data = asarray(f_image)
            v_images += [f_data] 
            v_timestamps += [f[0]]
        v_images = np.array(v_images)
        v_timestamps = np.array(v_timestamps)

        break

In [42]:
tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-uncased",
    use_fast=False,
    cache_dir="../.huggingface_cache/"
)

In [43]:
preprocess_SEND_files(
    modalities_data_dir,
    linguistic_tokenizer=tokenizer,
)

[[2043, 1045, 2001, 7763, 100, 2034, 2051, 1045, 2001, 1999, 2293], [2005, 100, 1045, 2228, 1045, 2196, 11333, 100, 2196, 2001, 1999, 2293], [2077, 100, 2008, 2001, 1996, 2034, 2051, 1045, 2001, 1999, 2293, 2007, 2619, 2008, 2001, 2066, 100, 1045], [3984, 2116, 2111, 2113, 2008, 100, 1045, 3246, 2017, 100], [1045, 2074, 2514, 2066, 100, 2061, 100], [1997, 2673, 2003, 2061, 100, 2066, 2074, 2156, 2008, 2711, 2066, 2045, 2017, 100, 2115], [2540, 2003, 2066, 6012, 1037, 3634, 2335, 1037, 2117], [1998, 100, 2017, 2514, 2061, 100, 100, 1045, 100, 100, 100, 2066, 100, 2017, 100], [1996, 3110, 1997, 100, 100], [100, 2055, 1037, 100, 2066, 100, 2200, 100, 2138, 1045, 2001, 2061, 100, 1045, 2001, 2014, 2767], [100, 1045, 2001, 2066, 100, 1045, 2196, 2371, 1999, 2293, 2007, 2619, 2030, 2018, 1037, 3276], [2077, 100, 2061, 2009, 2001, 2785, 1997, 100, 100, 2066, 100, 3100, 1045, 2064], [2991, 1999, 3062, 1045, 1045, 2064, 2991, 1999, 2293, 100, 1998], [2031, 2619, 1999, 2026, 2166, 2008, 2001, 10